In [2]:
import requests as req
import pandas as pd
import json
import time

# Utils

## Consts

In [3]:
GITHUB_GRAPHQL_URL  = 'https://api.github.com/graphql'
TOKEN = ''
HEADERS = {
  'Authorization': f'bearer {TOKEN}',
  'Content-Type': 'application/json'
}

## Functions

In [4]:
def measure_graphql(data : json)-> json:
    start_time = time.time()
    response = req.post(GITHUB_GRAPHQL_URL, headers=HEADERS, json=data)
    end_time = time.time()
    if response.status_code == 200:
        return end_time - start_time, len(response.content)

    raise Exception(f'Erro ao fazer requisição: {response.status_code} \n {response.text}')

## Queries

In [5]:
query_1 = '''
{
  user(login: "Netflix") {
    repositories(first: 5) {
      nodes {
        name
        url
      }
    }
  }
}
'''

query_2 = '''
{
  repository(owner: "Netflix", name: "Hystrix") {
    issues(states: OPEN, first: 5) {
      nodes {
        title
        url
      }
    }
  }
}
'''

query_3 = '''
{
  repository(owner: "Netflix", name: "Hystrix") {
    pullRequests(first: 5) {
      nodes {
        title
        url
        state
        mergedAt
        author {
          login
        }
        comments(first: 5) {
          nodes {
            body
          }
        }
      }
    }
  }
}
'''

query_gigante = '''
{
  repository(owner: "Netflix", name: "Hystrix") {
    pullRequests(first: 5) {
      nodes {
        title
        url
        state
        createdAt
        updatedAt
        mergedAt
        closedAt
        number
        additions
        deletions
        changedFiles
        author {
          login
          url
        }
        assignees(first: 5) {
          nodes {
            login
            url
          }
        }
        labels(first: 5) {
          nodes {
            name
            color
          }
        }
        milestone {
          title
          dueOn
          description
        }
        comments(first: 5) {
          nodes {
            body
            author {
              login
              url
            }
            createdAt
            updatedAt
          }
        }
        commits(first: 5) {
          nodes {
            commit {
              message
              committedDate
              url
              author {
                name
                email
                date
              }
            }
          }
        }
        files(first: 5) {
          nodes {
            path
            additions
            deletions
          }
        }
        reviews(first: 5) {
          nodes {
            body
            author {
              login
              url
            }
            state
            submittedAt
          }
        }
      }
    }
  }
}
'''



# experiment

In [6]:
data = []
for _ in range(10):
    graphql_time, graphql_size = measure_graphql(data={'query': query_1})
    data.append({
        'type_query': '1',
        'graphql_time': graphql_time,
        'graphql_size': graphql_size,
    })

for _ in range(10):
    graphql_time, graphql_size = measure_graphql(data={'query': query_2})
    data.append({
        'type_query': '2',
        'graphql_time': graphql_time,
        'graphql_size': graphql_size,
    })

for _ in range(10):
    graphql_time, graphql_size = measure_graphql(data={'query': query_3})
    data.append({
        'type_query': '3',
        'graphql_time': graphql_time,
        'graphql_size': graphql_size,
    })


for _ in range(10):
    graphql_time, graphql_size = measure_graphql(data={'query': query_gigante})
    data.append({
        'type_query': '4',
        'graphql_time': graphql_time,
        'graphql_size': graphql_size,
    })
    
    
df = pd.DataFrame(data)
df.to_csv('./api_grafh_results.csv', index=False)